In [ ]:
# Setup cell 1: clone the master's thesis github repository in case it does not exit
import os

repo_name = "masters_thesis"
parent_dir = os.path.basename(os.path.abspath(".."))

if not os.path.exists(repo_name) and repo_name not in parent_dir:
    !git clone https://github.com/danimue/masters_thesis.git
    %cd masters_thesis
    print("Sucessfully cloned Github repo")
    print("Changed directory to /masters_thesis")
else:
    print("Github repo already exists. Proceed with notebook execution.")

In [4]:
!pip install vllm triton

from IPython.display import clear_output
clear_output(wait=False)

Current working directory: /home/daniel-mueller92/Projects/masters_thesis
Script location: /home/daniel-mueller92/Projects/masters_thesis/utils.py
ARC data complete.


In [ ]:
from utils import check_dataset, load_files_from_json, json_to_string, create_prompt

In [ ]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=8000)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

llm = LLM(model="Qwen/Qwen2.5-7B-Instruct-AWQ", quantization="awq")